In [ ]:
# Comment (2023.01.18.)

# .py & .ipynb 차이점
# - MQpy/old_notebooks/pg_03_3_processing.ipynb 에서 시작.
# - 복붙 x. 타이핑 해가며 기억 깨우기...!
# - directory 달라져서 변경해야함. raw_files 폴더 위치 파일 사용하기.
# .py 일 경우, 파이썬 패키지 설치하기 위해서 환경변수 설정해야. 파이썬 설치해야.
# .ipynb 일 경우, 기본 탑재.


# Scope

# - proteinGroups.txt 로드 후 다음의 column data를 필터함.
# : contaminants, reverse, only identified site, uniquepeptide = 1 entries
# - protein names, Best MS/MS 항목에서 세미콜론(;) 구분된 데이터 요소를 split.
# - filter가 된 것을 *_base.xlsx 로 저장함.


# Uniprot API

# - IdMapping: https://uniprot.org/help/id_mapping 참조


# proteinGroups.txt 의 Dtype

# - object(delimeter), int64, float64


# vscode 터미널에서 pip 안될때

# 원인: python 옳게 설치했더라도 윈도우 클래스에서 pip 위치를 알지 못하기에 발생.
# python3 부터는 pip 내장이므로 python.exe 디렉터리 연결하면 됨.
# C:\Users\Simon\AppData\Local\Programs\Python\Python39\Scipts 를 다음에 추가하면됨.
# (고급 시스템 설정 보기) - (환경변수 탭) - (사용자에 대한 사용자 변수 및 시스템 변수의 path에 추가)


# 아나콘다 업데이트
# conda update -n base conda <-- base 업데이트
# conda update --all <-- 패키지 업데이트
# python -m pip install --upgrade pip <-- pip 패키지 업데이트
# conda --version <-- version 확인


# 다른 설정들
# 셀 라인 표시 shift + L

In [8]:
# Neccessary packages
import os
import pandas as pd
os.chdir('C:/Users/simhc/Documents/Github/proteome-tool/')

# Base_filter를 사용할 txt 베이스 raw 파일을 로드합니다.
global txtpath
txtpath = './raw/TestSample/QC_DU145/proteinGroups.txt'

# **kwargs 인수로 받는 함수... 클래스에 들어가면 init 잘 해야.
def isDrop(**kwargs):
    # **kwargs: keyword argument 줄임말. 인수를 딕셔너리로 받음.
    for key, value in kwargs.items():
        tmp1 = len(df)
        tmp2 = len(df[df[key] == value])
        ratio = (100*tmp2)/tmp1
        # key에 해당하는 value를 가진 entry 드랍.
        df.drop(df[df[key] == value].index, inplace=True)
        # value가 숫자인 경우: column 드랍하지 않음.
        # e.g., Razor + unique peptides의 경우, value = 1 인 entry 드랍 하지만, 나머지 entry는 남아야하므로.
        if isinstance(value, str):
            df.drop(columns=[key], inplace=True)
            print('message! >>> '+str(tmp2)+' (%.2f%%) entries were dropped, [' %ratio +key+'] column removed.')
        else:
            print('message! >>> '+str(tmp2)+' (%.2f%%) entries were dropped. ['%ratio +key+' = '+str(value)+']')
            # print('message! >>> '+str(tmp2)+' (%.2f%%) entries were dropped, ['+key+'] column removed' %ratio) <- 이 구문은 동작안함. %~~ 이게 string으로 나눈 같은 구역에 있어햐 함.
    return print('message! >>> '+str(len(df))+' entries left.')

# Split할 column 이름을 tuple (c1, c2)로 주고, delimeter의 기본값은 세미콜론(;)으로 되어있다.
def split_items(*args, delimeter=';'):
    for arg in args:
        tmp_series = pd.Series(df[arg])
        for ele in tmp_series:
            tmp = ele.split(delimeter)[0]
            tmp_series.replace(ele, tmp, inplace=True)
        print('message! >>> ['+arg+'] elements were splitted')
    return None
    # return df.head

def execute(drop_rule, split_cnames):
    # read txt file.
    isDrop(**drop_rule)
    split_items(*split_cnames)
    return None

if __name__ == "__main__":
    df = pd.read_table(filepath_or_buffer=txtpath, index_col=False)
    base_filter = {'Potential contaminant':'+', 'Reverse':'+', 'Only identified by site':'+', 'Razor + unique peptides':1}
    split_cnames = ('Protein IDs', 'Best MS/MS')
    execute(base_filter, split_cnames)

    # reset index
    df.reset_index(drop=True, inplace=True)


message! >>> 11 (0.92%) entries were dropped, [Potential contaminant] column removed.
message! >>> 12 (1.01%) entries were dropped, [Reverse] column removed.
message! >>> 25 (2.13%) entries were dropped, [Only identified by site] column removed.
message! >>> 385 (33.60%) entries were dropped. [Razor + unique peptides = 1]
message! >>> 761 entries left.
message! >>> [Protein IDs] elements were splitted
message! >>> [Best MS/MS] elements were splitted


In [9]:
# requests module 사용하여 web request-repond 프로세스를 수행합니다.
# uniprot accession 형식의 Protein IDs 를 Uniprot ID Mapping에 요청-응답 수행.
# 데이터프레임의 protein/gene name 및 sequence 등 항목을 수정해야 합니다.

# 참고페이지
# Uniprot API Help

import csv
from jobs.api_requests import uniprot_requests as unireq


def get_data_frame_from_tsv_results(tsv_results):
    reader = csv.DictReader(tsv_results, delimiter="\t", quotechar='"')
    return pd.DataFrame(list(reader))



# 판다스로 데이터프레임의 컬럼 데이터 뽑아냅니다.
# 테스트를 위하여 데이터의 일부를 새로운 객체 idSeries에 할당합니다.
PRids = pd.Series(df['Protein IDs'])
PRids2 = PRids[:6]



# 테스트 데이터가 담긴 객체를 uniprot_request.py 모듈의 execute 메소드 호출 및 링크 받아옵니다.
link = unireq.execute(PRids2)

# link 통해 나온 결과는 stream.. 말그대로 텍스트의 엄청난 스트림!!
# 같은 모듈의 get_id_mapping_results_stream 함수로 이 텍스트를 디코딩해서 필요한 부분을 추출합니다.
#
# ?compressed= true/false 는 다운로드 할 때 상관 있어서 차이는 없는 것으로 보임.
# 추출: %2C[식별자] (e.g., %2Crequest, %2Cprotein_name, %2Cgene_name 등등.) 의 규칙으로 추출해서
# 결과: &format=[확장자] (e.g., ~~&format=tsv) 로 결과물 만듦. delimeter 사용해서 데이터프레임 만든다.

tsv_rst = unireq.get_id_mapping_results_stream(str(link)+'?compressed=true&fields=accession%2Creviewed%2Cid%2Cprotein_name%2Cgene_names%2Clength%2Csequence&format=tsv')
tmp = get_data_frame_from_tsv_results(tsv_rst)
tmp.head()

Retrying in 3s
Retrying in 3s


,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Length,Sequence
0,A0FGR8,A0FGR8,reviewed,ESYT2_HUMAN,Extended synaptotagmin-2 (E-Syt2) (Chr2Syt),ESYT2 FAM62B KIAA1228,921,MTANRDAALSSHRHPGCAQRPRTPTFASSSQRRSAFGFDDGNFPGL...
1,A6NHR9,A6NHR9,reviewed,SMHD1_HUMAN,Structural maintenance of chromosomes flexible...,SMCHD1 KIAA0650,2005,MAAADGGGPGGASVGTEEDGGGVGHRTVYLFDRREKESELGDRPLQ...
2,Q99613,Q99613,reviewed,EIF3C_HUMAN,Eukaryotic translation initiation factor 3 sub...,EIF3C EIF3S8,913,MSRFFTTGSDSESESSLSGEELVTKPVGGNYGKQPLLLSEDEEDTK...
3,E9PAV3,E9PAV3,reviewed,NACAM_HUMAN,Nascent polypeptide-associated complex subunit...,NACA,2078,MPGEATETVPATEQELPQPQAETAVLPMSSALSVTAALGQPGPTLP...
4,O00148,O00148,reviewed,DX39A_HUMAN,ATP-dependent RNA helicase DDX39A (EC 3.6.4.13...,DDX39A DDX39,427,MAEQDVENDLLDYDEEEEPQAPQESTPAPPKKDIKGSYVSIHSSGF...


In [10]:
print(link)

https://rest.uniprot.org/idmapping/uniprotkb/results/ebd732214f1fa3b46504d64ad8382cb1b37a2cf0


In [ ]:
# Uniprot respond 데이터로 Protein names, Gene Names 교체할 수 있다.
# 엑셀작업시 순서대로 였으면 순서대로 해도 되지만
# 혹시 순서가 달라졌을수도 있으므로 From 값을 기준으로 replace를 수행할 것.

# (1) protein names replace
# 이건 거의 교차검증. 연산 너무 많다. (미완성)

def prot_name_replace_1():
    tmp_names = pd.Series(tmp['Protein names'])

    prot_ids = pd.Series(df['Protein IDs'])
    prot_names = pd.Seires(df['Protein names'])
    return None

# (2) 순서 같다는 것이 확실하면 column replace or append
# 별일 없으면 순서는 같다. 단, indicies 일치해야.
def prot_name_replace_2():
    df['Protein names'] = tmp['Protein names']
    df['Gene names'] = tmp['Gene Names']

    return df

tmp = prot_name_replace_2()
tmp.head

# prot_name_replace_2().head

In [ ]:
# 이 위로 완료.

# 필요한 열만 추출해서 output 만들기 (2023.01.19.)
# *.xlsx

# 현재 df에 속하는 column 이름들 list로 저장.
now_df_columns = list(df)
print(now_df_columns)

In [ ]:
import os
s = os.getcwd()

print(os.getcwd().replace('jobs',''))



In [ ]:
# 남겨놓을 column label 제외 필터링.
# 일반적인 base 파일에서 rough하게 이정도면 충분. (free TMT&SILAC. For only QC samples)

# (고급) 단백질에 해당하는 펩타이드 sequence 가 gene에서 시작 위치가 어디인지.
# Peptide IDs로 부터 peptides.txt 연계 해야.

# 전체 columns 불러와서
# *args 로 만들고 기본값은 None으로 해둔 뒤
# 남길 column을 추가해놓으면 완성.
import os
import pandas as pd
from time import localtime, strftime

rest = ['Protein IDs',
        'Protein names',
        'Gene names',
        'Razor + unique peptides',
        'Unique sequence coverage [%]',
        'Mol. weight [kDa]',
        'Sequence length',
        'Q-value',
        'Score',
        'Intensity',
        'id',
        'Peptide IDs',
        'Evidence IDs',
        'Best MS/MS']

# now_df_columns 안의 ele 들 중 ele에 대해서 if ele가 rest에 없는 것에 대해서 drop한다.
# 아래와 비교해서 뭐가 좋을까
def create_base_df1():
        filt = [ele for ele in now_df_columns if ele not in rest]
        df.drop(columns=filt, inplace=True)
        return None

# 남길 column 이름을 리스트로 받아옴.
def create_base_df2(c_names):
        base_df = df[c_names].copy()
        return base_df

def create_xlsx(dataframe):
        ntm = strftime('%Y%m%d_%H%M%S', localtime())
        cwd = os.getcwd()
        file_path ='..\\output\\ProteinGroups_base'+ntm+'.xlsx'
        dataframe.to_excel(excel_writer=file_path, index=False, encoding='utf-8')
        print('message! >>> file created. '+cwd.replace('jobs','')+file_path[3:])
        return None

# *.csv 도 엑셀로 열면되기 때문에 csv가 좀 더 보편적일지도... 다시 load할 때도.
def create_csv(dataframe):
        ntm = strftime('%Y%m%d_%H%M%S', localtime())
        cwd = os.getcwd()
        file_path ='..\\output\\ProteinGroups_base'+ntm+'.csv'
        dataframe.to_csv(path_or_buf=file_path, sep=',', index=False, encoding='utf-8')
        print('message! >>> file created. '+cwd.replace('jobs','')+file_path[3:])
        return None

# path 만드는 class 만들기.
# raw file에 따라서 달라져야해


new_df = create_base_df2(rest)
create_csv(new_df)

In [ ]:
# Base 파일 생성하기.
# 1단계로, protein groups에 대해서만 생성하고 2단계로 여러 txt 파일도 가능해야 합니다.
# tab/comma 로 구분되는 csv, utf-8 encoding, index=False
# 여기까지하면 예전에 작업한 pg_03_3_processing은 완료되었다 볼 수 있습니다.

# pg_03_3_processing.ipynb 에서 class 참조.
import pandas as pd
from time import localtime, strftime

class op_address():
    # __init__ 에서 초기 설정 값 잡는게 일반적임.
    def __init__(self, vers):
        self.vers = vers
        
    def pg_address(self):
        address = './outputs/'+txtpath[26:66]+'_proteinGroups_v'+str(self.vers)+'.txt'
        return address

    def pep_address(self):
        address = './outputs/'+txtpath[26:66]+'_peptides_v'+str(self.vers)+'.txt'
        return address
    
    def pg_address2(self):
        tm = localtime()
        ntm = strftime('%Y%m%d_%H%M%S', tm)
        address = './outputs/proteinGroups_v'+str(self.vers)+'-'+ntm+'.txt'
        return address
    
    def pep_address2(self):
        tm = localtime()
        ntm = strftime('%Y%m%d_%H%M%S', tm)
        address = './outputs/peptides_v'+str(self.vers)+'-'+ntm+'.txt'
        return address



# Generate file: version 1
v = op_address()
df.to_csv(path_or_buf=v.pg_address(), sep='\t', index=False, encoding='utf-8')
print('message! >>> '+v.pg_address()+' added.')

# df.to_excel

df.info()

In [ ]:
# ; 구분 요소 split (구버전)

# Protein IDs, Best MS/MS 의 delimeter (;) 로 구분된 데이터를 나누어 첫 번째 항목을 저장합니다.
# 해당 column 이름을 1차원 series로 저장하고, deilmeter를 기준으로 구분한 뒤,
# 첫번째 값으로 replace 합니다.
# split에 사용한 객체는 초기화 되어야 합니다.

# (1) Protein IDs
prot = pd.Series(df['Protein IDs'])
for ele in prot:
    tmp = ele.split(';')[0]
    prot.replace(ele, tmp, inplace=True)
print('message! >>> [Protein IDs] splitted.')

# (2) Best MS/MS
bmsms = pd.Series(df['Best MS/MS'])
for ele in bmsms:
    tmp = ele.split(';')[0]
    bmsms.replace(ele, tmp, inplace=True)
print('message! >>> [Best MS/MS] splitted.')


In [ ]:
# filter (구버전)

# Drop: Positive value for (Potential contaminant), (Reverse), (Only identified by site)
# Drop : 1 (Razor + unique peptides)

# Drop: Positive value for (Potential contaminant), and drop the column
len_tmp = len(df[df['Potential contaminant'] == '+'])
df.drop(df[df['Potential contaminant'] == '+'].index, inplace = True)
df.drop(columns=['Potential contaminant'], inplace=True)
print('message! >>> '+str(len_tmp)+' Potential contaminant entries were dropped')


# Drop: Positive value for (Reverse), and drop the column
len_tmp = len(df[df['Reverse'] == '+'])
df.drop(df[df['Reverse'] == '+'].index, inplace = True)
df.drop(columns=['Reverse'], inplace=True)
print('message! >>> '+str(len_tmp)+' [Reverse entries] were dropped')


# Drop: Positive value for (Only identified by site), and drop the column
len_tmp = len(df[df['Only identified by site'] == '+'])
df.drop(df[df['Only identified by site'] == '+'].index, inplace = True)
df.drop(columns=['Only identified by site'], inplace=True)
print('message! >>> '+str(len_tmp)+' [Only identified by site] entries were dropped')


# Drop: (Razor + unique peptides = 1), and drop the column
len_tmp = len(df[df['Razor + unique peptides'] == 1])
df.drop(df[df['Razor + unique peptides'] == 1].index, inplace = True)
df.drop(columns=['Razor + unique peptides'], inplace=True)
print('message! >>> '+str(len_tmp)+' [Razor + uniqe peptides = 1] (%.1f%%) entries were dropped' %(100*len_tmp/(len(df)+len_tmp)))


# Monitor: # of remained entries
print('message! >>> '+str(len(df))+' entries were remained')


# reset index
df.reset_index(drop=True, inplace=True)